In [8]:
!docker pull qdrant/qdrant

Using default tag: latest


error during connect: Post "http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/v1.46/images/create?fromImage=qdrant%2Fqdrant&tag=latest": open //./pipe/dockerDesktopLinuxEngine: The system cannot find the file specified.


In [11]:
import subprocess
import os
command = [
    "docker", "run", 
    "-d",  # Detach mode
    "-p", "6333:6333", 
    "-p", "6334:6334", 
    "-v", f"{os.getcwd()}/qdrant_storage:/qdrant/storage:z", 
    "qdrant/qdrant"
]

# Run the command in the background
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)



In [12]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [13]:
# from qdrant_client.models import Distance, VectorParams
# # Only run this when you want to reinitilize the collection. This will delete all nodes

# collection_name= "training_examples"
# client = QdrantClient(url="http://localhost:6333")
# exists=client.collection_exists(collection_name=collection_name)
# if exists:
#     client.delete_collection(collection_name=collection_name)
# client.create_collection(collection_name=collection_name,
#                         vectors_config=VectorParams(size=1024, distance=Distance.COSINE))

True

In [14]:
import json
import os

# Define the path to the JSON file
#go back a directory then go to the Dataset folder and then to the R2R_train_updated.json file
json_file_path= os.path.join(os.getcwd(), "..", "Dataset", "R2R_train_updated.json")

# Read the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Initialize a list to store combined instructions
formatted_strings = []

# Create the formatted string for each item and add it to the list
for item in data:
    if item["goals"]== None:
        continue
    formatted_string_1 = f"""
goal: {item['goals'][0]}

Answer: {item['instructions'][0]}
"""
    formatted_string_2 = f"""
goal: {item['goals'][1]}
Answer: {item['instructions'][1]}
"""
    formatted_string_3 = f"""
goal: {item['goals'][2]}
Answer: {item['instructions'][2]}
"""
    formatted_strings.append(formatted_string_1)
    formatted_strings.append(formatted_string_2)
    formatted_strings.append(formatted_string_3)

# Print the list of combined instruction strings
for formatted_string in formatted_strings:
    print(formatted_string)



goal: I am here inside the room and want to navigate to the Hallway.

Answer: Walk past the large picture and chair. Walk past the dining room table turn right into the hallway and stop. 


goal: I am standing in the hallway and want to navigate to a point beyond the clear glass table with the white chairs.
Answer: Go straight and pass the bar with the chair/stools then pass the clear glass table with the white chairs and turn right. Wait in that hallway. 


goal: I am near the painting of the mermaid and want to navigate to the Hallway past the Counter on the right.
Answer: With the painting of the mermaid towards your right, head straight. After passing the counter towards your right, turn right and wait. 


goal: I am here, near the painting, and want to navigate to the Hallway past the red painting and the doorway to the left of it.

Answer: Walk past the painting and turn right. Walk toward the red painting and through the doorway to the left of it. Stop by the chair. 


goal: I 

In [15]:
print("Number of instructions: ", len(formatted_strings))

Number of instructions:  360


In [16]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core import Settings
from dotenv import load_dotenv
load_dotenv()

embedding = CohereEmbedding(
    model="embed-english-v3.0",
    api_key=os.getenv("COHERE_API_KEY")
)

Settings.embed_model = embedding


vector_store = QdrantVectorStore(client=client, collection_name="training_examples", show_progress=True)
storage_context=StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(
vector_store,
storage_context=storage_context,
    )

In [17]:
from llama_index.core.schema import TextNode
nodes=[]
for combined_instruction in formatted_strings:
    node = TextNode(text=combined_instruction)
    nodes.append(node)


In [18]:
# import time

# #insert in batches of 50
# # Have to do this to prevent api rate limits
# # Only run if you have different nodes to insert
# for i in range(0, len(nodes), 50):
#     print(f"Inserting batch {i} to {i+50}")
#     index.insert_nodes(nodes[i:i+50])
#     time.sleep(10)


Inserting batch 0 to 50
Inserting batch 50 to 100
Inserting batch 100 to 150
Inserting batch 150 to 200
Inserting batch 200 to 250
Inserting batch 250 to 300
Inserting batch 300 to 350
Inserting batch 350 to 400
